# Setup

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Apps setting both CustomTextKey and Description in a ValidationIssue

In [ ]:
with Apps.init() as apps:
    apps.select(
        {
            "Validation issues": lambda app: app.cs.flat_map(
                lambda cs: cs.object_creations(type="ValidationIssue", fields=["CustomTextKey", "Description"])
            )
        }
    ).where(lambda app: app["Validation issues"].length > 0).table()

## IFormDataValidator.HasRelevantChanges

In [ ]:
with Apps.init() as apps:
    apps.where(lambda app: app.env == "prod").select(
        {
            "HasRelevantChanges": lambda app: app.cs.flat_map(
                lambda cs: cs.class_declarations(implements=["IFormDataValidator"])
            ).flat_map(
                lambda formDataValidator: formDataValidator.method_declarations(
                    name="HasRelevantChanges", returns="bool", modifiers=["public"]
                )
            )
        }
    ).where(lambda app: app["HasRelevantChanges"].length > 0).table()

## Apps implementing IUserAction on 8.6 or above

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.cs.some(lambda cs: cs.class_declarations(implements=["IUserAction"]).is_not_empty)
    ).group_by({"Env": lambda app: app.env, ">= 8.6.0": lambda app: app.backend_version > "8.5"}).select(
        {"Count": lambda apps: apps.length}
    ).table()

## IUserAction implementations with cancellation token

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.cs.some(
            lambda cs: cs.class_declarations(implements=["IUserAction"])
            .flat_map(lambda c: c.method_declarations(name="HandleAction"))
            .is_not_empty
        )
    ).group_by(
        {
            "Env": lambda app: app.env,
            "Uses cancellation token": lambda app: app.cs.flat_map(
                lambda cs: cs.class_declarations(implements=["IUserAction"]).flat_map(
                    lambda c: c.method_declarations(name="HandleAction", parameter_types=["CancellationToken"])
                )
            ).is_not_empty,
        }
    ).select(
        {"Count": lambda apps: apps.length}
    ).table()